# Подгрузка пакетов и данных

In [ ]:
cd ..

In [ ]:
import pandas as pd
import numpy as np
import json
from tqdm.notebook import tqdm

from gensim.models.word2vec import Word2Vec

from shared.consts import FUNCTIONS_JSON_PROPERTIES_NAMES, PATH_TO_FUNCTIONS_STORAGE_JSON, PATH_TO_WORD2VEC_MODEL

Загрузим уже построенную модель Word2Vec

In [ ]:
word2VecModel = Word2Vec.load(PATH_TO_WORD2VEC_MODEL)

Загрузим JSON, в котором мы сложили названия функций (и названия их арггументов)

In [ ]:
functions_storage_dictionary = json.load(open(PATH_TO_FUNCTIONS_STORAGE_JSON))

# Отделение объявления функций от их использования

Функции с одним и тем же названием могут быть объявлены по-разному, поэтому объявления функций будут склдываться в словарь по принципу:

<div style="text-align: center; font-weight: bold;">Имя репозитория + имя функции</div>

In [ ]:
SEPARATOR_REPO_FUNCNAME = '___'

def get_key_in_func_creation_dict(func_name, file_path):
    paths_segments = file_path.split('/')
    
    if (len(paths_segments) > 2):
        return paths_segments[1] + SEPARATOR_REPO_FUNCNAME + func_name
    else:
        return func_name
    
def get_func_name_from_key(key_in_func_creation_dict):
    return key_in_func_creation_dict.split(SEPARATOR_REPO_FUNCNAME)[-1]

In [ ]:
def find_all_functions_creations(func_storage_dic):
    func_creation_dic = {}
    
    for filePath in func_storage_dic.keys():
        for function in func_storage_dic[filePath]:
            if (
                function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_TYPE] == 'FunctionDeclaration' or
                function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_TYPE] == 'FunctionExpression'
               ):
                func_name = function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME]
                func_creation_dict_key = get_key_in_func_creation_dict(func_name, filePath)
                if func_creation_dict_key in func_creation_dic:
                    pass
                    #print(func_creation_dict_key, 'уже есть в словаре')
                func_creation_dic[func_creation_dict_key] = {
                    f"{FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES}" : function[FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES],
                    f"{FUNCTIONS_JSON_PROPERTIES_NAMES.FILE_PATH}": filePath
                }
    
    return func_creation_dic

In [ ]:
func_creation_dic = find_all_functions_creations(functions_storage_dictionary)

А теперь находим все ВЫЗОВЫ функций и смотрим, насколько их аргументы отличны объявления функции

In [ ]:
def find_all_func_executions(func_storage_dic, func_creation_dic, onlyInDictionary = True):
    func_exec_storage = []
    
    for filePath in func_storage_dic.keys():
        for function in func_storage_dic[filePath]:
            func_name = function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME]
            func_creation_dict_key = get_key_in_func_creation_dict(func_name, filePath)
            
            if (
                function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_TYPE] == 'CallExpression' and # должна быть вызовом функции
                len(function[FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES]) > 0 and # должна иметь хоть один аргумент
                (func_creation_dict_key in func_creation_dic or not onlyInDictionary)# должна быть в нашем словаре
            ):
                function[FUNCTIONS_JSON_PROPERTIES_NAMES.FILE_PATH] = filePath
                func_exec_storage.append(function)
    
    return func_exec_storage

In [ ]:
func_exec_storage = find_all_func_executions(functions_storage_dictionary, func_creation_dic)

In [ ]:
print('Размер словаря с объявлениями функций: ', len(func_creation_dic))
print('Размер массива с вызовами функций: ', len(func_exec_storage))

Словарь с объявлениями функций - словарь, в котором ключ - название функции, а значение – массив с названиями переданных аргументов.

Пример: функция с названием **confirmDelete**

In [ ]:
func_creation_dic[get_key_in_func_creation_dict('confirmDelete', 'scripts/Block8/PHPCI/public/assets/js/phpci.js')]

Массив с вызовами функций состоит из словарей следующего формата

In [ ]:
list(filter(lambda func_exec: func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME] == 'sleep',func_exec_storage))

# Алгоритм проверки схожести аргументов

In [ ]:
def checkIfWordInWord2VecDict(word, word2VecVocabulary = word2VecModel.wv):
    return word in word2VecVocabulary

def get_args_similarity(exec_func_name, exec_func_args_names, func_path, func_dict, word2VecModel):
    """Определяет схожесть названий аргументов в вызове функции с названиями аргументов в объявлении функции"""
    args_similarity = []
    
    func_in_func_creation_dic_key = get_key_in_func_creation_dict(exec_func_name, func_path)
    
    for argIndex in range(len(exec_func_args_names)):
        
        if argIndex > len(func_dict[func_in_func_creation_dic_key][FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES]) - 1:
            args_similarity.append(None)
            continue
        
        exec_arg = exec_func_args_names[argIndex]
        create_arg = func_dict[get_key_in_func_creation_dict(exec_func_name, func_path)][FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES][argIndex]
        
        if len(exec_arg) <= 3 or len(create_arg) <= 3 or '$' in exec_arg or '$' in create_arg:
            args_similarity.append(None)
            continue
        
        if (checkIfWordInWord2VecDict(exec_arg) and checkIfWordInWord2VecDict(create_arg)):
            args_similarity.append(word2VecModel.wv.similarity(exec_arg, create_arg))
        else:
            args_similarity.append(None)
        
    return args_similarity

Посмотрим, как это работает на примере

In [ ]:
def find_func_exec_and_compare_with_func_creation(funcName):
    example_func_execs = list(
        filter(lambda func: func[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME].endswith(funcName), func_exec_storage)
    )
    for example_func_exec in example_func_execs:
        func_name = example_func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME]
        func_path = example_func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.FILE_PATH]
        args_names = example_func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES]

        print('Вызов функции:', func_name, 'c аргументами', args_names)
        func_in_func_creation_dic_key = get_key_in_func_creation_dict(func_name, example_func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.FILE_PATH])
        print('Функция же была объявлена с словаре с аргументами:', func_creation_dic[func_in_func_creation_dic_key][FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES])

        print(get_args_similarity(func_name, args_names, func_path, func_creation_dic, word2VecModel))
        print('_________________________________________________')

In [ ]:
find_func_exec_and_compare_with_func_creation('updateLine')

Хорошо видно, что был пропущен второй аргумент **text**

In [ ]:
word2VecModel.wv.n_similarity(
    ['lastLine', 'lastSpans', 'estimateHeight'],
    ['line', 'text', 'markedSpans', 'estimateHeight']
)

другой пример

In [ ]:
find_func_exec_and_compare_with_func_creation('windowSearch')

# Реализация на всем массиве вызовов функции

Идея максимально проста. Проходим по массиву вызовов функции.

- По названию функции определяем в словаре с объявлениями функций, с какими названиями аргументов была объявлена эта функция
- Для каждой пары аргументов (название аргумента в момент вызовы функции и название аргумента в момент объявление функции) считаем близость их названий в векторной форме с помощью уже построенной модели word2Vec (это будет значение 0 до 1)
- Если эта схожесть меньше заданного порога (threshold), то выбрасываем предупреждение пользователю

### Threshold

In [ ]:
MIN_SIMILARITY_THRESHOLD = 0.1

Порог не должен быть большой, потому что модели важно достичь высокого precision, а не recall.

Если наш алгоритм будет часто беспокоить пользователя по пустякам, предупреждая об ошибках там, где их нет (false positive), то нашим алгоритмом просто никто не будет пользоваться. Нужно беспокоить программиста только прям в самых вероятных случаях наличия ошибки. Ведь программисты люди ленивые и занятые... 

In [ ]:
for func_exec in func_exec_storage:
    func_name = func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME]
    args_names = func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES]
    func_path = func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.FILE_PATH]
    
    if len(args_names) < 2:
        continue
    
    args_similarities = get_args_similarity(func_name, args_names, func_path, func_creation_dic, word2VecModel)
    is_below_threshold_args_similarities = list(map(lambda x: x and x < MIN_SIMILARITY_THRESHOLD, args_similarities))
    
    if (any(is_below_threshold_args_similarities)):
        print('В файле', func_path, 'проверь:')
        print('Была вызвана функция', func_name, 'с аргументами', args_names)
        print('Но вот объявлена функция была со следующими аргументами:')
        print(func_creation_dic[get_key_in_func_creation_dict(func_name, func_path)][FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES])
        print(args_similarities)
        print('_____________')

## Эксперименты с моделями

### Трансформируем данные

Оставим только функции, у которых 2 аргумента

In [ ]:
func_creation_dic_2_args = dict(
    filter(lambda elem: len(elem[1][FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES]) == 2, func_creation_dic.items())
)

In [ ]:
class TWO_ARGS_FUNC_COLUMNS:
    FIRST_ARG = 'arg1'
    SECOND_ARG = 'arg2'
    FUNC_NAME = 'funcName'
    TARGET = 'isBuggy'  # В верном ли порядке расставлены аргументы в функции

In [ ]:
two_args_funcs = pd.DataFrame.from_dict(func_creation_dic_2_args, orient='index')
two_args_funcs[[TWO_ARGS_FUNC_COLUMNS.FIRST_ARG, TWO_ARGS_FUNC_COLUMNS.SECOND_ARG]] = pd.DataFrame(
    two_args_funcs.argumentsNames.tolist(), index=two_args_funcs.index
)

two_args_funcs[TWO_ARGS_FUNC_COLUMNS.FUNC_NAME] = list(
    map(lambda rowName: get_func_name_from_key(rowName), two_args_funcs.index)
)
two_args_funcs.drop(['argumentsNames', 'filePath'], axis='columns', inplace=True)
two_args_funcs[TWO_ARGS_FUNC_COLUMNS.TARGET] = 0

two_args_funcs.reset_index(inplace=True, drop=True)

print('Количество строк:', len(two_args_funcs))
two_args_funcs.head(3)

Меняем местами аргументами и добавляем эти строчки как isBuggy = 1

(то есть получим 50% одного класса и 50% другого класса)

In [ ]:
buggy_code_rows = []

for rowIndex in tqdm(range(len(two_args_funcs))):
    new_row = {
        TWO_ARGS_FUNC_COLUMNS.FUNC_NAME: two_args_funcs.loc[rowIndex, TWO_ARGS_FUNC_COLUMNS.FUNC_NAME],
        TWO_ARGS_FUNC_COLUMNS.FIRST_ARG: two_args_funcs.loc[rowIndex, TWO_ARGS_FUNC_COLUMNS.SECOND_ARG],
        TWO_ARGS_FUNC_COLUMNS.SECOND_ARG: two_args_funcs.loc[rowIndex, TWO_ARGS_FUNC_COLUMNS.FIRST_ARG],
        TWO_ARGS_FUNC_COLUMNS.TARGET: 1
    }
    buggy_code_rows.append(new_row)

two_args_funcs = two_args_funcs.append(pd.DataFrame(buggy_code_rows)).reset_index(drop=True)

In [ ]:
print('Количество строк:', len(two_args_funcs))
two_args_funcs.tail(3)

Теперь заменяем слова на их векторное представление из word2Vec.

Если например, каждое слово представлено в модели word2Vec как вектор из 300 значений, то название функции + 2 аргумента + 1 целевая перееменная (isBuggy) = 901 переменная в датасете

In [ ]:
def replace_string_column_with_vectors(pd_df, columnName, word2VecVocabulary = word2VecModel.wv):
    DEFAULT_VALUE_IF_NOT_EXITS_IN_WORD2VEC = '0'
    
    vector_size = len(word2VecVocabulary[DEFAULT_VALUE_IF_NOT_EXITS_IN_WORD2VEC])
    
    future_columns_names = []
    
    vectorized_df = pd.DataFrame()
    
    for i in range(vector_size):
        future_columns_names.append(columnName + str(i))
    
    for rowIndex in tqdm(range(len(pd_df))):
        word = pd_df.loc[rowIndex, columnName]
        
        if not checkIfWordInWord2VecDict(word, word2VecVocabulary):
            #word = DEFAULT_VALUE_IF_NOT_EXITS_IN_WORD2VEC
            newColumnsDf = pd.DataFrame(np.array([None] * vector_size).reshape(-1, vector_size), columns = future_columns_names)
        else:
            newColumnsDf = pd.DataFrame(
                word2VecVocabulary[word].reshape(-1, len(word2VecVocabulary[word])), columns = future_columns_names
            )
   
        vectorized_df = vectorized_df.append(newColumnsDf)
    
    return pd.concat([pd_df, vectorized_df.reset_index(drop=True)], axis=1)

In [ ]:
two_args_funcs = replace_string_column_with_vectors(two_args_funcs, TWO_ARGS_FUNC_COLUMNS.FIRST_ARG, word2VecModel.wv)
two_args_funcs = replace_string_column_with_vectors(two_args_funcs, TWO_ARGS_FUNC_COLUMNS.SECOND_ARG, word2VecModel.wv)
two_args_funcs = replace_string_column_with_vectors(two_args_funcs, TWO_ARGS_FUNC_COLUMNS.FUNC_NAME, word2VecModel.wv)

two_args_funcs = two_args_funcs.drop(
    [TWO_ARGS_FUNC_COLUMNS.FIRST_ARG, TWO_ARGS_FUNC_COLUMNS.SECOND_ARG, TWO_ARGS_FUNC_COLUMNS.FUNC_NAME],
    axis=1
)

In [ ]:
two_args_funcs = two_args_funcs.dropna().reset_index(drop=True)

In [ ]:
len(two_args_funcs)

### Обучаем

Кормим это дело модели

In [ ]:
y = np.array(two_args_funcs[TWO_ARGS_FUNC_COLUMNS.TARGET])

X = two_args_funcs.drop(TWO_ARGS_FUNC_COLUMNS.TARGET, axis = 1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 1000, random_state = 27)

rf.fit(X, y);

### Проверяем на обучающей

In [ ]:
from sklearn import metrics

y_train_pred = rf.predict(X)
print('accuracy:', metrics.accuracy_score(y, y_train_pred))
print('recall:', metrics.recall_score(y, y_train_pred))
print('precision:', metrics.precision_score(y, y_train_pred))

### Проверяем на тестовой выборке

Формируем тестовую выборку из массива с вызовами функциями

Вновь фильтруем только функции с 2 аргументами

In [ ]:
func_exec_storage_all = find_all_func_executions(functions_storage_dictionary, func_creation_dic, False)

func_exec_storage_2_args = list(filter(lambda func: len(func[FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES]) == 2, func_exec_storage_all))

превращаем в датафрейм

In [ ]:
func_exec_2_args_df = pd.DataFrame(func_exec_storage_2_args)

func_exec_2_args_df = func_exec_2_args_df.rename(
    columns={FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME: TWO_ARGS_FUNC_COLUMNS.FUNC_NAME}
)

func_exec_2_args_df[[TWO_ARGS_FUNC_COLUMNS.FIRST_ARG, TWO_ARGS_FUNC_COLUMNS.SECOND_ARG]] = pd.DataFrame(
    func_exec_2_args_df[FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES].tolist(), index=func_exec_2_args_df.index
)

func_exec_2_args_df = func_exec_2_args_df.drop([
    FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_TYPE,
    FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES
], axis='columns')

In [ ]:
func_exec_2_args_df.head(3)

векторизуем перемененные 

In [ ]:
func_exec_2_args_df = replace_string_column_with_vectors(func_exec_2_args_df, TWO_ARGS_FUNC_COLUMNS.FIRST_ARG, word2VecModel.wv)
func_exec_2_args_df = replace_string_column_with_vectors(func_exec_2_args_df, TWO_ARGS_FUNC_COLUMNS.SECOND_ARG, word2VecModel.wv)
func_exec_2_args_df = replace_string_column_with_vectors(func_exec_2_args_df, TWO_ARGS_FUNC_COLUMNS.FUNC_NAME, word2VecModel.wv)

func_exec_2_args_df = func_exec_2_args_df.dropna()

INFO_COLUMNS = [FUNCTIONS_JSON_PROPERTIES_NAMES.FILE_PATH, TWO_ARGS_FUNC_COLUMNS.FIRST_ARG, TWO_ARGS_FUNC_COLUMNS.SECOND_ARG, TWO_ARGS_FUNC_COLUMNS.FUNC_NAME]

test_2_args_exec_df = func_exec_2_args_df.drop(
    INFO_COLUMNS,
    axis='columns'
)

func_exec_2_args_df = func_exec_2_args_df[INFO_COLUMNS]

In [ ]:
func_exec_2_args_df.head(3)

In [ ]:
test_2_args_exec_df.head(3)

In [ ]:
PRED_PROB_COLUMN = TWO_ARGS_FUNC_COLUMNS.TARGET + '_pred'

func_exec_2_args_df[PRED_PROB_COLUMN] = list(map(lambda x: x[1], rf.predict_proba(test_2_args_exec_df) ))

Посмотрим, как распределены вероятности уверенности модели, что конкретный вызов функции - баг.

In [ ]:
import seaborn as sns
sns.set(color_codes=True, rc={'figure.figsize':(11.7,8.27)})

sns.distplot(func_exec_2_args_df[PRED_PROB_COLUMN]);

Нам важно, что было была высокая precision (пусть даже и при низком recall). То есть мы не хотим докучать разработчика лишними false positives (нашей моделью тогда просто никто не будет пользоваться)

In [ ]:
pd.set_option('display.max_rows', 500)

func_exec_2_args_df[func_exec_2_args_df[PRED_PROB_COLUMN] >= 0.7]

Теперь нам осталось пройти по предсказанным моделью как ошибочным вызовам функций и понять, а действительно ли они ошибочные